# Implementing The Four And Five Factor Models

In [7]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

1- Specify the risky asset and the time horizon

In [4]:
RISKY_ASSET = 'AMZN'
START_DATE = '2013-12-31'
END_DATE = '2018-12-31'

2- Download the risk factors from prof. French's website

In [8]:
# three factors
df_three_factor = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=START_DATE)[0]
df_three_factor.index = df_three_factor.index.format()

# momentum factor
df_mom = web.DataReader('F-F_Momentum_factor', 'famafrench', start=START_DATE)[0]
df_mom.index = df_mom.index.format()

# five factors
df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=START_DATE)[0]
df_five_factor.index = df_five_factor.index.format()

3- Download the data of the risky asset from Yahoo Finance

In [9]:
asset_df = yf.download(RISKY_ASSET, start=START_DATE, end=END_DATE)
print(f'Download {asset_df.shape[0]} rows of data')

[*********************100%%**********************]  1 of 1 completed
Download 1258 rows of data


4- Calculate monthly returns

In [11]:
y = asset_df['Adj Close'].resample('M').last().pct_change().dropna()
y.index = y.index.strftime('%Y-%m')
y.name = 'return'

5- Merge the datasets for the four-factor model

In [12]:
# join all datasets on the index
four_factor_data = df_three_factor.join(df_mom).join(y)

# rename columns
four_factor_data.columns = ['mkt', 'smb', 'hml', 'rf', 'mom', 'rtn']

# divide everything (except returns) by 100
four_factor_data.loc[:, four_factor_data.columns != 'rtn'] /= 100

# select period of interest
four_factor_data = four_factor_data.loc[START_DATE:END_DATE]

# calculate excess returns
four_factor_data['excess_rtn'] = four_factor_data.rtn - four_factor_data.rf

four_factor_data.head()

,mkt,smb,hml,rf,mom,rtn,excess_rtn
2014-01,-0.0332,0.0089,-0.0207,0.0,0.0171,-0.100554,-0.100554
2014-02,0.0465,0.0035,-0.0031,0.0,0.0207,0.009507,0.009507
2014-03,0.0043,-0.0181,0.0493,0.0,-0.0329,-0.071058,-0.071058
2014-04,-0.0019,-0.0419,0.0117,0.0,-0.0389,-0.095847,-0.095847
2014-05,0.0206,-0.0188,-0.0013,0.0,0.0087,0.027685,0.027685


6- Merge the datasets for the five-factor models

In [14]:
# join all datasets on the index
five_factor_data = df_five_factor.join(y)

# rename columns
five_factor_data.columns = ['mkt', 'smb', 'hml', 'rmw', 'cma', 'rf', 'rtn']

# divide everything (except returns) by 100
five_factor_data.loc[:, five_factor_data.columns != 'rtn'] /= 100

# select period of interest
five_factor_data = five_factor_data.loc[START_DATE:END_DATE]

# calculate excess returns
five_factor_data['excess_rtn'] = five_factor_data.rtn - five_factor_data.rf

five_factor_data.head()

,mkt,smb,hml,rmw,cma,rf,rtn,excess_rtn
2014-01,-0.0332,0.0058,-0.0207,-0.0388,-0.0142,0.0,-0.100554,-0.100554
2014-02,0.0465,0.0014,-0.0031,-0.0023,-0.0045,0.0,0.009507,0.009507
2014-03,0.0043,-0.0109,0.0493,0.0213,0.0192,0.0,-0.071058,-0.071058
2014-04,-0.0019,-0.0412,0.0117,0.0348,0.0102,0.0,-0.095847,-0.095847
2014-05,0.0206,-0.0189,-0.0013,0.0005,-0.0100,0.0,0.027685,0.027685


7- Estimate the four factor model

In [16]:
four_factor_model = smf.ols(formula='excess_rtn ~ mkt + smb + hml + mom', data=four_factor_data).fit()

print(four_factor_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     16.90
Date:                Mon, 19 Feb 2024   Prob (F-statistic):           4.30e-09
Time:                        16:50:50   Log-Likelihood:                 86.372
No. Observations:                  60   AIC:                            -162.7
Df Residuals:                      55   BIC:                            -152.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0093      0.008      1.162      0.2

8- Estimate the five factor model

In [17]:
five_factor_model = smf.ols(formula='excess_rtn ~ mkt + smb + hml + rmw + cma', data=five_factor_data).fit()

print(five_factor_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.557
Method:                 Least Squares   F-statistic:                     15.86
Date:                Mon, 19 Feb 2024   Prob (F-statistic):           1.36e-09
Time:                        16:54:03   Log-Likelihood:                 89.429
No. Observations:                  60   AIC:                            -166.9
Df Residuals:                      54   BIC:                            -154.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0099      0.008      1.297      0.2